# YNK forecast

## Import data

In [2]:
# Import data in csv from github

import pandas as pd

# Import data from github
url = 'https://raw.githubusercontent.com/sagebhardt/forecast-AI-ynk/main/Data_Cleaned.csv'
df = pd.read_csv(url)
df.head()

/var/folders/7n/r8rrh9ss6q9b_lyhlmx1x_hw0000gn/T/ipykernel_83315/954859110.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,S,2022.0,Spring Summer,16519,1,24000,"20167,,,",NaN,NaN,NaN
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,L,2022.0,Spring Summer,16519,1,19990,"10083,,,",NaN,NaN,NaN
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,M,2022.0,Fall,16519,1,24990,"12602,,,",NaN,NaN,NaN
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,L,2022.0,Fall,16519,1,22990,"11596,,,",NaN,NaN,NaN
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,5T,2023.0,Spring,16519,1,24990,"12602,,,",NaN,NaN,NaN


## Clean data  

In [3]:
#Delete last three columns
df = df.drop(df.columns[[-1, -2, -3]], axis=1)
df.head()

,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Sub_Cat,Gender,Age_Group,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,Performance,Male,Adult,S,2022.0,Spring Summer,16519,1,24000,"20167,,,"
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,Performance,Female,Adult,L,2022.0,Spring Summer,16519,1,19990,"10083,,,"
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,Performance,Male,Adult,M,2022.0,Fall,16519,1,24990,"12602,,,"
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,Performance,Male,Adult,L,2022.0,Fall,16519,1,22990,"11596,,,"
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,Fashion,Male,Pre School,5T,2023.0,Spring,16519,1,24990,"12602,,,"


In [4]:
newdf = df['COGS'].map(lambda x: str(x)[:-3])
# Change the type of the column to float
df['COGS'] = newdf
df.head()


,Week,Store,Center,Type,Brand,SKU,SKU_Full,Description,Price,Denom.tipo mat.,...,Sub_Cat,Gender,Age_Group,Size,Season_Year,Season,PPL_Count,SUM_Q,Revenue,COGS
0,W09,01-Ahumada,1000,ZAPP,AD,ADHB0042,ADHB004200S,POLERON HODDIE DARK GREY T/ S,39990,Apparel,...,Performance,Male,Adult,S,2022.0,Spring Summer,16519,1,24000,20167
1,W09,01-Ahumada,1000,ZAPP,AD,ADHE4926,ADHE492600L,POLERA WMS BASIC LOGO LIGH BLUE T / L,19990,Apparel,...,Performance,Female,Adult,L,2022.0,Spring Summer,16519,1,19990,10083
2,W09,01-Ahumada,1000,ZAPP,AD,ADHK7092,ADHK709200M,SHORT BBALL T/M,24990,Apparel,...,Performance,Male,Adult,M,2022.0,Fall,16519,1,24990,12602
3,W09,01-Ahumada,1000,ZAPP,AD,ADHL6934,ADHL693400L,POLERA M CAMO T/ L,22990,Apparel,...,Performance,Male,Adult,L,2022.0,Fall,16519,1,22990,11596
4,W09,01-Ahumada,1000,ZAPP,AD,ADHR9501,ADHR950105T,POLERA LB DY SM T T/5T,24990,Apparel,...,Fashion,Male,Pre School,5T,2023.0,Spring,16519,1,24990,12602


In [5]:
# Get column types
df.dtypes

# Change the type of the column to int
#newdf = df['COGS'].astype(int)

Week                object
Store               object
Center               int64
Type                object
Brand               object
SKU                 object
SKU_Full            object
Description         object
Price                int64
Denom.tipo mat.     object
2da_Hierarchy       object
3_Hierarchy         object
4th_Hierarchy       object
Sub_Cat             object
Gender              object
Age_Group           object
Size                object
Season_Year        float64
Season              object
PPL_Count            int64
SUM_Q                int64
Revenue              int64
COGS                object
dtype: object

# PCA Analysis

In [8]:
import numpy as np

max_chunk = 100

print('Data shape =',df.shape)

max_text_len = df['Description'].apply(lambda x:len(x)).max()
print('max text length = ',max_text_len)

Data shape = (134325, 23)
max text length =  40


In [9]:
def get_max_len(tokenized):
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    return max_len

def process_chunk(dtf_chunk):
    import transformers as ppb
    import torch
    
    #model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    
    tokenized = dtf_chunk['Description'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    
    max_len = get_max_len(tokenized)
    
    if (max_len > 512):
        print('Chunk has',max_len,'tokens,..., Aborting.')
        features = -1
    else: 
        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)
        attention_mask = torch.tensor(attention_mask)
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)
        features = last_hidden_states[0][:,0,:].numpy()
    
    return features


def get_chunks(max_text_len, chunk_len, dtf):

    import transformers as ppb
   
    #model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    
    cuts = np.arange(0,max_text_len,chunk_len)
    token_dic ={}
    max_len_dic = {}
    chunk_dic = {}
    i = 0
    max_all = 0
    for c in cuts:
     
        name = 'chunk_'+str(i)
        chunk_dic[name] = pd.DataFrame(dtf['Description'].apply(lambda x: x[c:c+chunk_len]))
        token_dic[name] = dtf['Description'].apply((lambda x: tokenizer.encode(x[c:c+chunk_len], add_special_tokens=True)))
        max_len_dic[name] = get_max_len(token_dic[name])
        if (max_len_dic[name]>max_all):
            max_all = max_len_dic[name]
        print(name,', starting at character',c,'has len',max_len_dic[name])
        i = i + 1
    print(i,'Chunks created with max len =',max_all)   
    
    return chunk_dic, max_all

In [10]:
chunk_len = 1000
chunk_dic, max_all = get_chunks(max_text_len, chunk_len, df)

chunk_0 , starting at character 0 has len 24
1 Chunks created with max len = 24


In [11]:
feature_dic = {}
for key,value in chunk_dic.items():
    features = process_chunk(value)
    feature_dic[key] = features
    print('Finished processing', key)
    #break

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


: 

: 

In [ ]:
feature_dic

In [ ]:
feature_dic['chunk_0'].shape

## Calculate SVD

In [ ]:
features = feature_dic['chunk_0']
for key,value in chunk_dic.items():
    if (key!='chunk_0'):
        features = np.concatenate((features,feature_dic[key]),axis=1)
pd.DataFrame(features).to_csv('full_realbert_feat_additional_feedback.csv',header=False,index=False)
features.shape

In [ ]:
import matplotlib.pyplot as plt

u,s,vh = np.linalg.svd(features)
plt.plot(np.log(s))
#show grid lines
plt.grid()

#zoom in first 25 and make plot bigger

plt.figure(figsize=(10,10))
plt.plot(np.log(s[0:25]))
plt.grid()


In [ ]:
pd.DataFrame(u[:,0:10]).to_csv('Feat_REAL_BERT_Description.csv',index=False,header=False) 

print('Finished')  